# Datapipeline:

WARNING:

+ For the script to work the data files must be positiong in the data path of the repository. 
Or the path to the data must be adapted accordingly. In this script all csv data must be in data_path 
+ The *Data which is used* : EU_all_current_shared.parquet from Andreas 

# 0. Loading Data 

# 1. Main data pipeline 
Treat data to reach a point were we have the data frame ready for the separate treatments for classification and regression treatment 

* mapping of entries in countrys, and companies and so on, see Andreas jupyter notebooks 
* only unique values remains in columns which are categorical 

# 2. Data pipeline for classification models 
Prepare data so that it can be used for all basic classification models 

# 3. Data pipeline for regression models 
Prepare data so that it can be used for all basic regression models 


 * Retrieving data
 + Get an overview and count empty entries in the dataset  
 + Drop Fuel consumption because it is too much correlated with CO2 Emission 
 * Explanatory Variable is Ewltp (g/km) and Erwltp(g/km)
 + Filter all data that has no explanatory variable  
 + Drop data before 2015 because of VW diesel Scandal 
 + Drop data before 2018 because of the succesfull implementation of the new WLT measurement (which will be our target variable Ewltp)
 + Keep all contries, discuss why 
 + Search for unique Car Identification number to ommit double cars which have the same characterisics 
 + Discuss percentage error compared to the mean value for different car IDs: TAN, VFN, Ves
 + Filter out Ve that are not combination from T & Va = Ve 
 + Filter out rows which have ? in their Ve 

 + Drop Columns which are not relevant: ID, r
 + make separate DF with electric cars for separate research


# 4. Evaluation with baseline classification models 


# 5. Evaluation with baseline regression models 



